In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex31")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex31").getOrCreate()

25/01/09 12:13:50 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/09 12:13:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/09 12:13:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# le o arquivo txt e cria dataframe com uma unica coluna, a "value"
data = spark.read.text("/home/paloma/jupyter_spark/ex31/requests_ex31.txt")

In [3]:
data_ips = data.rdd
data_ips.collect()
# aqui cada linha = objeto Row

[Row(value='66.249.69.97 --[24/Sep/2014:22:25:44  +0000]  "GET http://www.google.com/bot.html”'),
 Row(value='66.249.69.97 --[24/Sep/2014:22:26:44  +0000]  "GET  http://www.google.com/how.html”'),
 Row(value='66.249.69.97 --[24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”'),
 Row(value='71.19.157.179 --[24/Sep/2014:22:30:12  +0000]  "GET http://www.google.com/faq.html”'),
 Row(value='66.249.69.95 --[24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”'),
 Row(value='66.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”'),
 Row(value='56.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”')]

In [4]:
data_ips = data_ips.map(lambda line: line.value) # obtem a string em value
data_ips.collect() # agora temos uma lista de strings e nao ded objetos Row

['66.249.69.97 --[24/Sep/2014:22:25:44  +0000]  "GET http://www.google.com/bot.html”',
 '66.249.69.97 --[24/Sep/2014:22:26:44  +0000]  "GET  http://www.google.com/how.html”',
 '66.249.69.97 --[24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”',
 '71.19.157.179 --[24/Sep/2014:22:30:12  +0000]  "GET http://www.google.com/faq.html”',
 '66.249.69.95 --[24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”',
 '66.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”',
 '56.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”']

In [5]:
# filtrando ips que possuem requisicao a google
data_ips_filtered = data_ips.filter(lambda word: 'google' in word)
data_ips_filtered.collect()

['66.249.69.97 --[24/Sep/2014:22:25:44  +0000]  "GET http://www.google.com/bot.html”',
 '66.249.69.97 --[24/Sep/2014:22:26:44  +0000]  "GET  http://www.google.com/how.html”',
 '71.19.157.179 --[24/Sep/2014:22:30:12  +0000]  "GET http://www.google.com/faq.html”',
 '66.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”',
 '56.249.69.97 --[24/Sep/2014:56:26:44  +0000]  "GET http://www.google.com/how.html”']

In [13]:
# map para recortar as strings, so parte com os ips
data_ips_unique = data_ips_filtered.map(lambda word: word[:13])
# outra forma de fazer seria separando as strings usando espacos, essa separacao quebra a string
# da mesma linha em varias e guarda numa lista, cada elemento da lista sera uma string, em seguida
# a cada iteracao do map eh obtido o primeiro elemento dessa lista, que eh o ip. Dessa forma evitamos
# pegar os espacos em branco que restam na forma usada acima.
# data_ips_unique = data_ips_filtered.map(lambda line: str.split(line, ' ')[0])
data_ips_unique.collect()

['66.249.69.97 ',
 '66.249.69.97 ',
 '71.19.157.179',
 '66.249.69.97 ',
 '56.249.69.97 ']

In [15]:
# removendo espacos em branco
data_ips_unique = data_ips_unique.map(lambda line: line.strip())
# pegando ips distintos
data_ips_unique = data_ips_unique.distinct()
data_ips_unique.collect()

['66.249.69.97', '71.19.157.179', '56.249.69.97']

In [17]:
output_path = "/home/paloma/jupyter_spark/ex31/output"
data_ips_unique.coalesce(1).saveAsTextFile(output_path)
spark.stop()